In [1]:
import requests
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
import numpy as np
import datetime
from datetime import date
import time
from urllib.request import Request, urlopen

import urllib.request
from bs4 import BeautifulSoup
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials

scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

credentials = {'CREDENTIALS_HERE'}
gc = gspread.service_account_from_dict(credentials)

In [2]:
ibc_workbook = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YYAkpgglEspNWhe4AD4O4KRZq4sa_HBQDVKdE1doUc4/edit#gid=0')
proj_2_df = pd.DataFrame(ibc_workbook.worksheet('Project 2 - Data').get_all_records())

In [3]:
data_df = proj_2_df.reindex(['Hubspot ID', 'Company name', 'Contact Name', 'Contact Phone', 'Contact Email'], axis = 1)

In [4]:
data_df

,Hubspot ID,Company name,Contact Name,Contact Phone,Contact Email
0,4108677698,160 Driving Academy,Person 1: Steve Gold: Founder,Person 1: (877) 350-0884,steve@160drivingacademy.com
1,4108671201,"1st Choice Career Institute, Inc.",Person 1: Dilya Khalitova: VP/Director,Person 1: (347) 230 - 5305,Dilya@1stchoicehcsi.com
2,4108648042,A T Still University of Health Sciences - Kirk...,Person 1: Craig Phelps: President\nPerson 2: R...,Person 1: (660) 626 - 2391,CPhelps@atsu.edu\nrRieder@atsu.edu
3,4108744253,A-Technical College,Person 1: Ali Bayrami: Campus President\nPerso...,Person 1: (213) 747 - 8292,abayrami@atechcollege.edu\nsmiranda@atechcolle...
4,4108658001,"AAA School of Dental Assisting, Inc.",Person 1: Anthony Balsam: Director,Person 1: (407) 324 - 0960,anthony@aaasoda.com
...,...,...,...,...,...
7340,4108664637,Yukon Beauty College Inc - Yukon,,,
7341,4108679019,Z Hair Academy - Overland Park,,,
7342,4104275479,Zane State College - Zanesville,,,
7343,6945771144,Zaytuna College - Berkeley,,,


In [5]:
all_contacts_df = pd.DataFrame()
data_df = proj_2_df[proj_2_df['Moved over by Sophie'] == 2]
data_df = data_df.reset_index().drop('index', axis=1)
data_df = data_df.reindex(['Hubspot ID', 'Company name', 'Contact Name', 'Contact Phone', 'Contact Email'], axis = 1)

for i in range(len(data_df)):
    # Name
    data_df['Contact Name'][i] = data_df['Contact Name'][i].split('\n')
    data_df['Contact Name'][i] = list(data_df['Contact Name'][i])

    # Phone
    data_df['Contact Phone'][i] = data_df['Contact Phone'][i].split('\n')
    data_df['Contact Phone'][i] = list(data_df['Contact Phone'][i])

    # Email
    data_df['Contact Email'][i] = data_df['Contact Email'][i].split('\n')
    data_df['Contact Email'][i] = list(data_df['Contact Email'][i])

    for j in range(len(data_df['Contact Name'][i])):
        # Name
        clean_person = re.sub(r'[0-9]', '', data_df['Contact Name'][i][j]).replace('Person :', '')
        new_name = clean_person.split(':')[0]
        try:
            role = clean_person.split(':')[1]
        except:
            role = ''
        # Phone
        try:
            clean_phone = data_df['Contact Phone'][i][j].replace('Person 1:', '').replace('Person 2:', '').replace('Person 4:', '').replace('NA', '')
        except:
            clean_phone = ''
        # Email
        try:
            clean_email = re.sub(r'[0-9]', '', data_df['Contact Email'][i][j]).replace('Person :', '').lower().replace('na', '')
        except:
            clean_email  = ''
        
        all_contacts_df = all_contacts_df.append({'Hubspot ID': data_df['Hubspot ID'][i], 'Company Name': data_df['Company name'][i], 'Contact Name': new_name, 'Contact Role':role, 'Contact Email': clean_email, 'Contact Phone':clean_phone}, ignore_index=True)
    

In [6]:
all_contacts_df['First Name'] = ''
all_contacts_df['Last Name'] = ''

for i in range(len(all_contacts_df)):
    try:
        all_contacts_df['First Name'][i] = all_contacts_df['Contact Name'][i].strip().split(' ')[0]
        all_contacts_df['Last Name'][i] = str(all_contacts_df['Contact Name'][i].strip().split(' ')[1:]).replace('[', '').replace(']', '').replace("'", '').replace(',', '')
    except:
        continue

In [7]:
all_contacts_df = all_contacts_df.reindex(['Hubspot ID', 'Company Name', 'First Name', 'Last Name','Contact Role', 'Contact Phone', 'Contact Email'], axis=1)

In [8]:
all_contacts_df

,Hubspot ID,Company Name,First Name,Last Name,Contact Role,Contact Phone,Contact Email
0,4.108678e+09,160 Driving Academy,Steve,Gold,Founder,(877) 350-0884,steve@drivingacademy.com
1,4.108711e+09,Midway Paris Beauty School - Ridgewood,Tiffany,Lebron,Administrator,,tiffany@midwayparis.com
2,6.946594e+09,Midway University - Midway,John,P. Marsden,President,(859) 846 - 5310,jmarsden@midway.edu
3,6.946594e+09,Midway University - Midway,John,Martin,Campus Minister,(859) 846 - 5391,john.martin@midway.edu
4,6.946583e+09,Midwest Barber College - Topeka,Randy,Niemeier,Co-Owner,(785) 266 - 2500,
...,...,...,...,...,...,...,...
3502,6.946594e+09,Yeshiva Gedolah Imrei Yosef D'spinka - Brooklyn,Joseph,Solomon,President,,
3503,6.945779e+09,Yeshiva Gedolah Keren Hatorah - Lakewood,Yehudah,Landau,CEO,(732) 942 - 1811,ylandau@kerenhatorah.org
3504,6.946520e+09,Yeshiva Gedolah Kesser Torah - Monsey,NA,,,,
3505,6.946552e+09,Yeshiva Gedolah of Cliffwood - Keyport,NA,,,,


In [9]:
all_contacts_df = all_contacts_df.drop_duplicates(subset=['First Name', 'Last Name', 'Contact Role', 'Contact Phone', 'Contact Email'], keep='last').reset_index().drop('index', axis=1)
all_contacts_df = all_contacts_df.rename({'Hubspot ID':'Company ID', 'Contact Role':'Job Title'}, axis = 1)

In [10]:
all_contacts_df

,Company ID,Company Name,First Name,Last Name,Job Title,Contact Phone,Contact Email
0,4.108678e+09,160 Driving Academy,Steve,Gold,Founder,(877) 350-0884,steve@drivingacademy.com
1,4.108711e+09,Midway Paris Beauty School - Ridgewood,Tiffany,Lebron,Administrator,,tiffany@midwayparis.com
2,6.946594e+09,Midway University - Midway,John,P. Marsden,President,(859) 846 - 5310,jmarsden@midway.edu
3,6.946594e+09,Midway University - Midway,John,Martin,Campus Minister,(859) 846 - 5391,john.martin@midway.edu
4,6.946583e+09,Midwest Barber College - Topeka,Randy,Niemeier,Co-Owner,(785) 266 - 2500,
...,...,...,...,...,...,...,...
2177,6.946624e+09,Yeshiva D'monsey Rabbinical College - Monsey,Rabbi,Schiff,President/CEO,(914) 356 - 1980,
2178,6.946567e+09,Yeshiva Derech Chaim - Brooklyn,Naftali,Miller,Director,,
2179,6.946594e+09,Yeshiva Gedolah Imrei Yosef D'spinka - Brooklyn,Joseph,Solomon,President,,
2180,6.945779e+09,Yeshiva Gedolah Keren Hatorah - Lakewood,Yehudah,Landau,CEO,(732) 942 - 1811,ylandau@kerenhatorah.org


In [12]:
assignment_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1X8XJ2gaIdtz8RtinovL-cdnU_H4b6MKyr9168EP8yCg/edit#gid=2078431139')
contacts_sheet = assignment_wb.worksheet('Contacts')

In [13]:
set_with_dataframe(contacts_sheet, all_contacts_df)